### Instituto Tecnologico de Costa Rica (ITCR)
### Sede Interuniversitaria de Alajuela
### Escuela de Computacion
### Curso: Inteligencia Artificial
### Estudiantes: 

 - Brandon Ledezma Fernández - 2018185574
 - Walter Morales Vásquez - 2018212846

### Profesora:
 
 - Maria Auxiliadora Mora


# Tarea Programada Número 4
---
#### Introducción:
En este trabajo práctico se aplicarán conceptos básicos de aprendizaje automático
utilizando redes neuronales recurrentes para resolver problemas que involucran el
procesamiento de lenguaje natural.

El o los estudiantes deberán realizar dos ejercicios. El primero consiste en implementar
una red neuronal recurrente aplicada a un problema de clasificación de textos de opinión
sobre prendas de vestir de mujer. El segundo ejercicio consiste en reconocer nombres de
entidades en textos (NER, Named-Entity Recognition)

El objetivo del trabajo es poner en práctica las habilidades de investigación y el
conocimiento adquirido durante el curso sobre redes neuronales por medio de
ejercicios prácticos que permitan al estudiante experimentar con el aprendizaje profundo. 

In [1]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split

## A. Clasificación   de   textos   con   redes   neuronales   recurrentes  LSTM utilizando Pytorch.

Se desea que, dado un comentario de revisión de una prenda de vestir, predecir la calificación dada por el comprador. La calificación toma valores enteros entre 1 y 5, donde 1 corresponde a la peor calificación y 5 a la mejor.

Datos: Utilice los datos de evaluación de prendas de vestir de mujer disponibles en Kaggle (nicapotato, 2018) para:

1. Cargue y prepare los datos para ser introducidos a la red LSTM.

In [226]:
nRowsRead = 1000

# Se carga el archivo con los datos solicitados (defaultofcredit.csv) y se define
# a la columna "default_payment_next_month" como la objetivo.
reviews = pd.read_csv('./data/Womens Clothing E-Commerce Reviews.csv', delimiter=',', nrows=nRowsRead)
#reviews = pd.read_csv('./data/a.csv')
reviews.describe()

,Unnamed: 0,Clothing ID,Age,Rating,Recommended IND,Positive Feedback Count
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,499.500000,910.282000,43.502000,4.194000,0.818000,2.614000
std,288.819436,198.919762,12.357374,1.080524,0.386038,5.700131
min,0.000000,2.000000,20.000000,1.000000,0.000000,0.000000
25%,249.750000,850.000000,35.000000,4.000000,1.000000,0.000000
50%,499.500000,907.000000,41.000000,5.000000,1.000000,1.000000
75%,749.250000,1060.000000,52.000000,5.000000,1.000000,3.000000
max,999.000000,1196.000000,93.000000,5.000000,1.000000,84.000000


In [227]:
# Delete missing observations for following variables
# for x in ["Clothing ID","Age","Title","Review Text","Rating","Recommended IND","Positive Feedback Count","Division Name","Department Name","Class Name"]:
#     reviews = reviews[reviews[x].notnull()]
    
# X = reviews.drop(columns = reviews.columns[3:5])
# X = pd.get_dummies(X)
# y = reviews['Review Text']
# y = [elem.split() for elem in y]

#reviews['Rating'] = reviews[(reviews['Rating'] >= 1)  (reviews['Rating'] <= 5)]

for x in ["Review Text","Rating"]:
     reviews = reviews[reviews[x].notnull()]

X = reviews['Review Text']
#X = [elem.split() for elem in X]
y = reviews['Rating']

# word_to_ix = {}
# for review in X:
# #     if review not in word_to_ix:
# #         word_to_ix[review] = len(word_to_ix)
#     for word in review:
#         if word not in word_to_ix:
#             word_to_ix[word] = len(word_to_ix)

ratings = [1, 2, 3, 4, 5]
            
X

0      Absolutely wonderful - silky and sexy and comf...
1      Love this dress!  it's sooo pretty.  i happene...
2      I had such high hopes for this dress and reall...
3      I love, love, love this jumpsuit. it's fun, fl...
4      This shirt is very flattering to all due to th...
                             ...                        
995    Received these as a christmas gift from my dau...
996    Every year around this time, our beloved retai...
997    I tried on this sweater in the store and immed...
998    Super soft and comfortable. runs a little larg...
999    Wear the collar down & it favors grace kelly. ...
Name: Review Text, Length: 958, dtype: object

In [229]:
pip3 install tensorflow

SyntaxError: invalid syntax (<ipython-input-229-6ea927cc6e88>, line 1)

In [228]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

sentence_sq = tokenizer.texts_to_sequences(X)

print(sentence_sq)

ModuleNotFoundError: No module named 'tensorflow'

2. Utilizando PyTorch defina una red recurrente LSTM para procesar el conjunto de datos y clasificar los comentarios de usuario. 

In [201]:
# def tensor_mean(input):
    
#     output = torch.tensor(np.zeros(input[0].size(0)))
#     #output = input.clone().detach()
#     #output = output[torch.arange(output.size(0))==0]
#     #print('output', output)
# #     print('output', output)
# #     print('output', output[0])
#     #print('input', input)
# #     print('input[0].size(0)', input[0].size(0))
    
#     for i in range(input.size(0)):
#         for j in range(input[i].size(0)):
#             output[j] += input[i][j]
            
#     for i in range(output.size(0)):
#         output[i] /= input.size(0)
    
#     return output.data.copy_(input.data)
#     return torch.tensor([output.tolist()], grad_fn=input.grad_fn)#requires_grad=False) grad_fn=input.grad

# # Funciones utilitarias

# def max_values(x):
#     # Retorna el valor máximo y en índice o la posición del valor en un vector x.
#     # Parámetros: 
#     #    x: vector con los datos. 
#     # Salida: 
#     #    out: valor 
#     #    inds: índice
#     out, inds = torch.max(x,dim=1)   
#     return out, inds

# tensor_mean(torch.tensor([[1,1],[2,2]]))
# #max_values(torch.tensor([[1,2,1,5,9]]))

RuntimeError: output with shape [2] doesn't match the broadcast shape [2, 2]

In [217]:
def tensor_mean(input):
    
    #output = torch.tensor(np.zeros(input[0].size(0)))
    print('input', input)
    #input?
    output = input.clone().detach()
    #output.grad_fn.data.copy_(input.grad_fn.data)
    
    input = input[torch.arange(input.size(0))==0].double()
#     print('output', output)
#     print('input', input)
#     print('output', output)
#     print('output', output[0])
#     print('input[0].size(0)', input[0].size(0))
    
    for i in range(1, output.size(0)):
        for j in range(output[i].size(0)):
            input[0][j] += output[i][j]
            
    for i in range(input[0].size(0)):
        input[0][i] /= output.size(0)
    return input
    #return torch.tensor([output.tolist()], grad_fn=input.grad_fn)#requires_grad=False) grad_fn=input.grad

# Funciones utilitarias

def max_values(x):
    # Retorna el valor máximo y en índice o la posición del valor en un vector x.
    # Parámetros: 
    #    x: vector con los datos. 
    # Salida: 
    #    out: valor 
    #    inds: índice
    out, inds = torch.max(x,dim=1)   
    return out, inds

print(tensor_mean(torch.tensor([[1,1],[3,3]])))
#max_values(torch.tensor([[1,2,1,5,9]]))

def clone_vs_deepcopy(x):
    import copy
    import torch

    x_clone = x.clone()
    x_deep_copy = copy.deepcopy(x)
    #
    x.mul_(-1)
    print(f'x = {x}')
    print(f'x_clone = {x_clone}')
    print(f'x_deep_copy = {x_deep_copy}')

input tensor([[1, 1],
        [3, 3]])
tensor([[2., 2.]], dtype=torch.float64)


In [223]:
# Definición del modelo

# El modelo es una clase que debe heredar de nn.Module
class LSTMTagger(nn.Module):
    
    # Incialización del modelo
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        # Primero se pasa la entrada a través de una capa Embedding. 
        # Esta capa construye una representación de los tokens de 
        # un texto donde las palabras que tienen el mismo significado 
        # tienen una representación similar.
        
        # Esta capa captura mejor el contexto y son espacialmente 
        # más eficientes que las representaciones vectoriales (one-hot vector).
        # En Pytorch, se usa el módulo nn.Embedding para crear esta capa, 
        # que toma el tamaño del vocabulario y la longitud deseada del vector 
        # de palabras como entrada. 
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # El LSTM toma word_embeddings como entrada y genera estados ocultos
        # con dimensionalidad hidden_dim.  
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # La capa lineal mapea el espacio de estado oculto 
        # al espacio de etiquetas
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        # Pase hacia adelante de la red. 
        # Parámetros:
        #    sentence: la oración a procesar
        embeds = self.word_embeddings(sentence)
        
        print('embeds', embeds)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        
        print('lstm_out', lstm_out)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        print('tag_space', tag_space)
        
        # Se utiliza softmax para devolver un peso por etiqueta
        tag_scores = F.log_softmax(tag_space, dim=1)
        print('tag_scores', tag_scores)
        print('torch.mean(tag_scores,0)', torch.mean(tag_scores,0))
        return torch.mean(tag_scores,0)

# Instanciación del modelo, definición de la función de pérdida y del optimizador   

# Hiperparámetros de la red
# Valores generalmente altos (32 o 64 dimensiones).
# Se definen pequeños, para ver cómo cambian los pesos durante el entrenamiento.

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

# Instancia del modelo
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(ratings)) # len([1,2,3,4,5]))

# Función de pérdida: Negative Log Likelihood Loss (NLLL). 
# Útil para problemas de clasificacion con C clases.
loss_function = nn.NLLLoss()

# Optimizador Stochastic Gradient Descent  
optimizer = optim.SGD(model.parameters(), lr=0.1)

3. Separe las muestras en datos de entrenamiento y evaluación y entrene el modelo. 

In [159]:
X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y.values.reshape(-1,1),
                                        train_size   = 0.7,
                                        random_state = 1234,
                                        shuffle      = False
                                    )

# print('X_train', X_train.values)
# print('X_test', X_test.values)
# print('y_train', y_train)
# print('y_test', y_test)

#print('X.values.tolist()', X.values.tolist())

#test = torch.tensor(y.values.tolist())

# X_train = torch.tensor(X_train.values.astype(np.float32)) 
# X_test = torch.tensor(X_test.values.astype(np.float32)) 

# y_train = torch.tensor(y_train.values.astype(np.float32))
# y_test = torch.tensor(y_test.values.astype(np.float32))

# train_target = torch.tensor(train['Target'].values.astype(np.float32))
# train = torch.tensor(train.drop('Target', axis = 1).values.astype(np.float32)) 
# train_tensor = data_utils.TensorDataset(train, train_target) 
# train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

print(X_train[0], y_train[0])
word_to_ix

['Absolutely', 'wonderful', '-', 'silky', 'and', 'sexy', 'and', 'comfortable'] [4]


{'Absolutely': 0,
 'wonderful': 1,
 '-': 2,
 'silky': 3,
 'and': 4,
 'sexy': 5,
 'comfortable': 6,
 'Love': 7,
 'this': 8,
 'dress!': 9,
 "it's": 10,
 'sooo': 11,
 'pretty.': 12,
 'i': 13,
 'happened': 14,
 'to': 15,
 'find': 16,
 'it': 17,
 'in': 18,
 'a': 19,
 'store,': 20,
 "i'm": 21,
 'glad': 22,
 'did': 23,
 'bc': 24,
 'never': 25,
 'would': 26,
 'have': 27,
 'ordered': 28,
 'online': 29,
 'petite.': 30,
 'bought': 31,
 'petite': 32,
 'am': 33,
 '5\'8".': 34,
 'love': 35,
 'the': 36,
 'length': 37,
 'on': 38,
 'me-': 39,
 'hits': 40,
 'just': 41,
 'little': 42,
 'below': 43,
 'knee.': 44,
 'definitely': 45,
 'be': 46,
 'true': 47,
 'midi': 48,
 'someone': 49,
 'who': 50,
 'is': 51,
 'truly': 52,
 'I': 53,
 'had': 54,
 'such': 55,
 'high': 56,
 'hopes': 57,
 'for': 58,
 'dress': 59,
 'really': 60,
 'wanted': 61,
 'work': 62,
 'me.': 63,
 'initially': 64,
 'small': 65,
 '(my': 66,
 'usual': 67,
 'size)': 68,
 'but': 69,
 'found': 70,
 'outrageously': 71,
 'small.': 72,
 'so': 73,
 '

In [224]:
# Preparación de los datos 
def prepare_sequence(seq, to_ix):
    # Prepara tensores de indices de palabras a partir de una oración.
    # Parámetros:
    #   seq: oración
    #   to_ix: diccionario de palabras.
    idxs = [to_ix[w] for w in seq]
    #idxs = to_ix[seq]
    return torch.tensor(idxs, dtype=torch.long)

# Entrenar el modelo 

# Valores antes de entrenar
# El elemento i, j de la salida es la puntuación entre la etiqueta j para la palabra i.
with torch.no_grad():
    inputs = prepare_sequence(X_train[0], word_to_ix)
    print('inputs', inputs)
    tag_scores = model(inputs)
    
    print(X_train[0])
    
    # Clasificación    
    print(tag_scores)

# Corridas o épocas
for epoch in range(500):  # definir epocas por fuera?       ## [N, D, N], [N, D, N]
    for sentence, current_ratings in zip(X_train, y_train): ## [1], [2]
        print('sentence', sentence)
        print('current_ratings', current_ratings)
        ## Paso 1. Pytorch acumula los gradientes.
        # Es necesario limpiarlos
        model.zero_grad()

        # Paso 2. Se preparan las entradas, es decir, se convierten a
        # tensores de índices de palabras.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        #print('current_ratings', current_ratings)
        #targets = prepare_sequence(rating, ratings)

        # Paso 3. Se genera la predicción (forward pass).
        tag_scores = model(sentence_in)
#         print('tag_scores', tag_scores)
        targets = torch.tensor(current_ratings)
        
#         print('aslkdmalksmdslakmdaslkmsadkl')
        #clone_vs_deepcopy(tag_scores)

        # Paso 4. se calcula la pérdida, los gradientes, y se actualizan los 
        # parámetros por medio del optimizador.
        #loss = loss_function(tag_scores, targets.long()).float()
        loss = loss_function(tag_scores, targets)
        print('loss', loss)
        loss.backward()
        optimizer.step()

# Despligue de la puntuación luego del entrenamiento
with torch.no_grad():
    inputs = prepare_sequence(training_data[0], word_to_ix)
    tag_scores = model(inputs)
   
    print("Resultados luego del entrenamiento para la primera frase")
    # Las palabras en una oración se pueden etiquetar de tres formas.
    # La primera oración tiene 4 palabras "El perro come manzana"
    # por eso el tensor de salida tiene 4 elementos. 
    # Cada elemento es un vector de pesos que indica cuál etiqueta tiene más
    # posibilidad de estar asociada a la palabra. Es decir hay que calcular 
    # la posición del valor máximo
    print(tag_scores)

inputs tensor([0, 1, 2, 3, 4, 5, 4, 6])
embeds tensor([[ 0.4107,  1.0144, -1.0657, -0.6897, -0.1951,  1.4534],
        [-1.0985, -0.6816, -0.5365,  0.0429,  0.1207,  0.4357],
        [ 0.8546, -0.8771,  1.0214,  0.1660,  0.4804, -1.0735],
        [ 1.4899,  1.4295, -0.4465,  1.4641, -0.2803,  1.0421],
        [ 3.2272,  0.4155,  0.5625, -0.4644, -1.0295,  0.9616],
        [-0.1536,  0.1546,  0.6190, -2.7387,  0.3251,  1.1198],
        [ 3.2272,  0.4155,  0.5625, -0.4644, -1.0295,  0.9616],
        [-1.4809, -1.6705,  0.5626, -1.2437, -0.3058,  1.1479]])
lstm_out tensor([[[-1.8970e-01, -6.6170e-02,  6.0442e-02,  3.7987e-02, -1.6331e-01,
           8.9354e-02]],

        [[-2.1501e-01, -1.4187e-02,  9.2862e-02,  2.5745e-02, -1.4446e-01,
           1.6180e-01]],

        [[ 2.1043e-01,  2.8561e-01, -3.8741e-02, -1.8480e-02, -9.2887e-02,
           5.8461e-02]],

        [[ 5.7229e-02,  4.7122e-01, -3.1438e-04,  1.2780e-02, -1.9538e-01,
           2.1837e-02]],

        [[ 5.8387e-03,  6.0

ValueError: Expected 2 or more dimensions (got 1)

4. Evalúe el modelo resultante utilizando una matriz de confusión y métricas extraídas a partir de esta (ie. precisión, exhaustividad y F1). 

5. Genere y documente sus conclusiones (incluya al menos cuatro conclusionesimportantes). 

Referencias
----------------
*

## B. Reconocimiento   de   nombres   de   entidades   (NER,   Name   EntityRecognition) con redes neuronales recurrentes utilizando Pytorch.

El reconocimiento de nombres de entidades (NER)  es  el proceso  de identificar y categorizar elementos clave (ej. entidades) en el texto. Una entidad puede ser cualquier palabra o secuencia de palabras que se refieren a una persona, animal, sitio o cosa (ej.empresa, región geográfica, objeto). Cada entidad detectada se clasifica en una categoría predeterminada. Normalmente, NER se aborda como un problema de etiquetado de secuencias. Una explicación muy detallada de porqué es importante extraer entidades de los textos se encuentra en (Monge, 2020).

Los algoritmos de extracción de entidades pueden únicamente detectar la presencia de una entidad y marcarla como tal o pueden detectar y clasificar cada entidad que encuentran.

Ejemplo: En una oración como “arbusto de 2 m. flores lila.”.  Cada palabra representa un token donde “arbusto” y “flores” son los elementos de interés a marcar. El etiquetado “token inicial- token interno” es una forma común de indicar dónde comienzan y terminan las entidades. En el ejemplo anterior la etiqueta sería “B O O O B O” donde B representael inicio de la entidad y O cualquier otro token. Para la oración “botones florales rosados.” la etiqueta estaría dada por “B I O” donde “B” marca el token inicio de la entidad e “I” los otros tokens que son parte de esta, es decir “B I” delimita la entidad “botones florales”. 

Otra forma de marcar y etiquetar, es además de delimitar la entidad, asignar a esta la clase a la que corresponde, por ejemplo: empresa, ciudad, persona, entre otros. Para el presente ejercicio se va a utilizar este enfoque.

Utilice los datos para reconocer y clasificar nombres de entidades compartidos en Kaggle por (Ranjan, 2020) para:

1. Cargue y prepare los datos para ser introducidos a la red recurrente.

In [12]:
nRowsRead = 1000

# Se carga el archivo con los datos solicitados (defaultofcredit.csv) y se define
# a la columna "default_payment_next_month" como la objetivo.
ner = pd.read_csv('./data/NER_Dataset.csv', delimiter=',', nrows=nRowsRead)

ner.describe()

,Sentence_ID,Word,POS,Tag
count,1000,1000,1000,1000
unique,1000,1000,997,877
top,Sentence: 10821,"['Grenada', 'was', 'seized', 'by', 'a', 'Marxi...","['NNP', 'POS', 'NNP', 'NNP', 'VBZ', '.']","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
freq,1,1,3,13


In [13]:
# Delete missing observations for following variables
# for x in ["Clothing ID","Age","Title","Review Text","Rating","Recommended IND","Positive Feedback Count","Division Name","Department Name","Class Name"]:
#     reviews = reviews[reviews[x].notnull()]
    
# X = reviews.drop(columns = reviews.columns[3:5])
# X = pd.get_dummies(X)
# y = reviews['Review Text']
# y = [elem.split() for elem in y]

#reviews['Rating'] = reviews[(reviews['Rating'] >= 1)  (reviews['Rating'] <= 5)]


# ner["Word"] = ner["Word"].str.replace(r"'", '', regex=True)
# ner["Word"] = ner["Word"].str.replace(r"[", '', regex=True)
# ner["Word"] = ner["Word"].str.replace(r"]", '', regex=True)
# ner["Word"] = ner["Word"].str.replace(r" ", '', regex=True)

# ner["Word"] = ner["Word"].str.replace(r"'", '', regex=True)
# ner["Word"] = ner["Word"].str.replace(r"[", '', regex=True)
# ner["Word"] = ner["Word"].str.replace(r"]", '', regex=True)
# ner["Word"] = ner["Word"].str.replace(r" ", '', regex=True)

#ner["Word"] = ner["Word"].Series.str.replace('[', '', regex=True)

for x in ["Word", "Tag"]:
#     ner[x] = ner[x].str.replace("[", '', 1, regex=True)
#     ner[x] = ner[x][-1].str.replace("]", '', 1, regex=True)
    ner[x] = ner[x].str.replace("'", '', regex=True)
    ner[x] = ner[x].str.replace(", ", ' ', regex=True)

In [14]:
X = ner['Word']
X = [elem[1:-1].split() for elem in X]
# X[0] = X[0][1:]
# X[-1] = X[0][1:]
y = ner['Tag']
y = [elem[1:-1].split() for elem in y]
# y.pop(0)
# y.pop()


# def limpiar_datos(dataset):
#     for i in range(len(dataset)):
#         dataset[i].remove()
    
word_to_ix = {}
for frase in X:
#     if review not in word_to_ix:
#         word_to_ix[review] = len(word_to_ix)
    for palabra in frase:
        if palabra not in word_to_ix:
            word_to_ix[palabra] = len(word_to_ix)

            
etiquetas_a_indice = {'O':0,'B-geo':1,'B-gpe':2,'B-org':3,'B-nat':4,'B-art':5,'B-eve':6,'B-tim':7,'B-per':8,
                      'I-geo':9,'I-gpe':10,'I-org':11,'I-nat':12,'I-art':13,'I-eve':14,'I-tim':15,'I-per':16}

In [15]:
flatten = lambda t: [item for sublist in t for item in sublist]
test = flatten(y)
# print(test)
myset = set(test)
print(myset)
len(word_to_ix)

{'B-geo', 'B-org', 'B-gpe', 'I-tim', 'B-tim', 'B-nat', 'I-art', 'O', 'B-eve', 'I-eve', 'I-nat', 'B-per', 'I-org', 'I-gpe', 'I-per', 'I-geo', 'B-art'}


4783

2. Utilizando PyTorch defina una red recurrente LSTM para procesar, localizar y clasificar las entidades presentes en el texto (como la vista en clase). 

In [16]:
# Definición del modelo

# El modelo es una clase que debe heredar de nn.Module
class LSTMTagger(nn.Module):
    
    # Incialización del modelo
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
 

        # Primero se pasa la entrada a través de una capa Embedding. 
        # Esta capa construye una representación de los tokens de 
        # un texto donde las palabras que tienen el mismo significado 
        # tienen una representación similar.
        
        # Esta capa captura mejor el contexto y son espacialmente 
        # más eficientes que las representaciones vectoriales (one-hot vector).
        # En Pytorch, se usa el módulo nn.Embedding para crear esta capa, 
        # que toma el tamaño del vocabulario y la longitud deseada del vector 
        # de palabras como entrada. 
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # El LSTM toma word_embeddings como entrada y genera estados ocultos
        # con dimensionalidad hidden_dim.  
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # La capa lineal mapea el espacio de estado oculto 
        # al espacio de etiquetas
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        # Pase hacia adelante de la red. 
        # Parámetros:
        #    sentence: la oración a procesar
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))

#         print('tag_space', tag_space)
        
        # Se utiliza softmax para devolver un peso por etiqueta
        tag_scores = F.log_softmax(tag_space, dim=1)
#         print('tag_scores', tag_scores)
        return tag_scores

# Instanciación del modelo, definición de la función de pérdida y del optimizador   

# Hiperparámetros de la red
# Valores generalmente altos (32 o 64 dimensiones).
# Se definen pequeños, para ver cómo cambian los pesos durante el entrenamiento.

EMBEDDING_DIM = 6#34
HIDDEN_DIM = 6#34

# Instancia del modelo
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(etiquetas_a_indice)) # len([1,2,3,4,5]))

# Función de pérdida: Negative Log Likelihood Loss (NLLL). 
# Útil para problemas de clasificacion con C clases.
loss_function = nn.NLLLoss()#nn.CrossEntropyLoss()#nn.NLLLoss()

# Optimizador Stochastic Gradient Descent  
optimizer = optim.SGD(model.parameters(), lr=0.1)

3. Separe las muestras en datos de entrenamiento y evaluación y entrene el modelo. 

In [17]:
proporcion = lambda : 0.7

# Se randomizan las posiciones de las clases.
random.shuffle(X, proporcion)
random.shuffle(y, proporcion)

indice = int(len(X)*0.3)

X_train, y_train = X[indice:], y[indice:]
X_test, y_test = X[:indice], y[:indice]

In [ ]:
# Preparación de los datos 
def prepare_sequence(seq, to_ix):
    # Prepara tensores de indices de palabras a partir de una oración.
    # Parámetros:
    #   seq: oración
    #   to_ix: diccionario de palabras.
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

# Entrenar el modelo 

# Valores antes de entrenar
# El elemento i, j de la salida es la puntuación entre la etiqueta j para la palabra i.
with torch.no_grad():
    inputs = prepare_sequence(X_train[0], word_to_ix)
    
#     print('inputs', inputs, inputs.size())
    
    tag_scores = model(inputs)
    
#     print(X_train[0])
    
    # Clasificación    
#     print(tag_scores)

# Corridas o épocas
for epoch in range(100):  
    
    if epoch % 10 == 0:
        print(epoch)
    
    for sentence, etiqueta in zip(X_train, y_train):
#         print('sentence', sentence)
#         print('tags', etiqueta)
        ## Paso 1. Pytorch acumula los gradientes.
        # Es necesario limpiarlos
        model.zero_grad()

        # Paso 2. Se preparan las entradas, es decir, se convierten a
        # tensores de índices de palabras.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(etiqueta, etiquetas_a_indice)
#         print('targets', targets)
        
        # Paso 3. Se genera la predicción (forward pass).
        tag_scores = model(sentence_in)
#         print('tag_scores', tag_scores)

        # Paso 4. se calcula la pérdida, los gradientes, y se actualizan los 
        # parámetros por medio del optimizador.
#         print('tag_scores', tag_scores, tag_scores.size())
#         print('targets', targets)
        loss = loss_function(tag_scores, targets)
#         print('lossmljsmaLKMSALKmaslkMLKSAMlkmaldksmlkads\nlksadmldksamlkdsmlakds', loss)
        loss.backward()
        optimizer.step()

In [21]:
# Despligue de la puntuación luego del entrenamiento
with torch.no_grad():
    inputs = prepare_sequence(X_train[0], word_to_ix)
    tag_scores = model(inputs)
   
    print("Resultados luego del entrenamiento para la primera frase")
    # Las palabras en una oración se pueden etiquetar de tres formas.
    # La primera oración tiene 4 palabras "El perro come manzana"
    # por eso el tensor de salida tiene 4 elementos. 
    # Cada elemento es un vector de pesos que indica cuál etiqueta tiene más
    # posibilidad de estar asociada a la palabra. Es decir hay que calcular 
    # la posición del valor máximo
    print(tag_scores)

Resultados luego del entrenamiento para la primera frase
tensor([[-7.3905e-03, -1.0090e+01, -1.5211e+01, -1.1010e+01, -7.2639e+00,
         -8.0030e+00, -9.4255e+00, -8.9662e+00, -1.3398e+01, -1.0983e+01,
         -1.0065e+01, -6.0938e+00, -7.5411e+00, -7.1470e+00, -1.0219e+01,
         -6.0572e+00, -9.9318e+00],
        [-2.3842e-07, -2.5576e+01, -3.8041e+01, -2.8248e+01, -2.2967e+01,
         -2.1987e+01, -2.5964e+01, -1.5428e+01, -3.4131e+01, -3.3666e+01,
         -2.8596e+01, -2.4236e+01, -2.1589e+01, -2.4282e+01, -2.8306e+01,
         -1.8465e+01, -3.2006e+01],
        [-7.4237e+00, -3.1427e+00, -4.7707e+00, -5.8448e-01, -6.1850e+00,
         -6.6674e+00, -6.3390e+00, -8.1915e+00, -9.6096e-01, -8.0502e+00,
         -7.2502e+00, -9.0914e+00, -8.7334e+00, -7.3305e+00, -7.8715e+00,
         -1.0492e+01, -9.2291e+00],
        [-1.7134e+01, -1.3331e+01, -8.7627e+00, -6.9044e+00, -7.2531e+00,
         -7.7772e+00, -8.5695e+00, -2.1454e+01, -6.9506e+00, -4.0762e+00,
         -5.8616e+00,

4. Evalúe el modelo resultante. Utilice la métrica propuesta por el InternationalWorkshop on Semantic Evaluation (SemEval), una explicación básica está disponible en (Batista, 2018).  

In [24]:
# Probar el modelo

# Funciones utilitarias

def max_values(x):
    # Retorna el valor máximo y en índice o la posición del valor en un vector x.
    # Parámetros: 
    #    x: vector con los datos. 
    # Salida: 
    #    out: valor 
    #    inds: índice
    out, inds = torch.max(x,dim=1)   
    return out, inds

def test_examples(test_data):

   with torch.no_grad():
      inputs = prepare_sequence(test_data, word_to_ix)
      tag_scores = model(inputs)
    
 
   print("FRASE") 
   print("La frase original", test_data)    
   print("La frase original preprocesada", inputs)
   print("Salida del modelo", tag_scores)
   print("Valores máximos e índices", max_values(tag_scores))    
    

#print("Índice de palabras")
#print("word_to_idx", word_to_ix)

print("Etiquetas")
print(etiquetas_a_indice)

#Frase 1
# Las palabras en una oración se pueden etiquetar de tres formas.
# La primera oración tiene 3 palabras "El perro juega"
# por eso el tensor de salida tiene 3 elementos. 
# Cada elemento es un vector de pesos que indica cuál etiqueta tiene más
# posibilidad de estar asociada a la palabra. Es decir hay que calcular 
# la posición del valor máximo. 
#   Ejemplo 1: "El perro juega" ["DET", "NN", "V"]
# Ejemplo: 0, 1, 2 {"DET": 0, "NN": 1, "V": 2} => DET, NN, V 
test_examples(X_test[0])

print(y_test[0])

Etiquetas
{'O': 0, 'B-geo': 1, 'B-gpe': 2, 'B-org': 3, 'B-nat': 4, 'B-art': 5, 'B-eve': 6, 'B-tim': 7, 'B-per': 8, 'I-geo': 9, 'I-gpe': 10, 'I-org': 11, 'I-nat': 12, 'I-art': 13, 'I-eve': 14, 'I-tim': 15, 'I-per': 16}
FRASE
La frase original ['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.']
La frase original preprocesada tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,  9, 15,  1,
        16, 17, 18, 19, 20, 21])
Salida del modelo tensor([[ 0.0000e+00, -2.8718e+01, -4.3295e+01, -2.6295e+01, -2.7822e+01,
         -2.4183e+01, -2.8142e+01, -2.2482e+01, -3.5736e+01, -3.1194e+01,
         -2.8209e+01, -2.7903e+01, -2.8850e+01, -2.7250e+01, -2.9109e+01,
         -2.0848e+01, -3.5956e+01],
        [ 0.0000e+00, -2.3958e+01, -3.7577e+01, -1.9667e+01, -2.7258e+01,
         -2.2015e+01, -2.7311e+01, -1.9343e+0

5. Genere   y   documente   sus   conclusiones   (incluya   al   menos   cuatro conclusiones importantes).

In [ ]:
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0]

Referencias
----------------
*